In [1]:
import d4rl
import time
import torch
import torch.nn as nn
import numpy as np
import gym
from matplotlib import pyplot as plt

from model import Resnet, EBMDiffusionModel
from diffusion import DiffusionModel
from data_sampler import DataSampler

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
No module named 'flow'
/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Nov 28 2023 23:45:17
/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/pybullet_envs/env_bases.py:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/kezhang/work/stochastic-critic/.venv/

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
env_name = 'halfcheetah-medium-v2'
env = gym.make(env_name)

dataset = d4rl.qlearning_dataset(env)

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 13.36it/s]


In [4]:
data_sampler = DataSampler(dataset, device)

In [5]:
sample = data_sampler.sample(keys=["observations", "actions"], batch_size=1, concat=True)

In [6]:
sample

tensor([[-0.0645, -0.1249, -0.0300,  0.2821, -0.2713, -0.4986, -0.1250, -0.2375,
          4.9944,  0.6616, -1.7421, -0.2686,  2.4481, -1.1449, 15.4297, -3.2255,
          2.1888, -0.9741, -0.9833, -0.9910, -0.4961, -0.9770,  0.8868]],
       device='cuda:0')

In [13]:
batch_size = 1000
data_dim = sample.shape[-1]

EMA = .999

n_steps = 100
net_params = {"n_layers": 4,
              "h_dim": 128,
              "emb_dim": 32}

In [14]:
# xr = [-.75, .75]
# yr = [-.75, .75]

# # load data
# dataset_energy, dataset_sample = toy_gmm(std=.03)
# x = dataset_sample(batch_size)

# plot_samples(x)
# plt.show()
# x = x.reshape(x.shape[0], -1)

In [15]:
net = Resnet(n_steps=n_steps, n_layers=4, x_dim=data_dim, h_dim=128, emb_dim=32)

ebm_net = EBMDiffusionModel(net)

ddpm = DiffusionModel(data_dim, n_steps, ebm_net, device=device, var_type="beta_forward").to(device)

In [16]:
def logp_unnorm(net, ddpm, x, t):
    scale_e = ddpm.energy_scale(-2 - t)
    t = torch.ones((x.shape[0],)).int() * t
    return -net.neg_logp_unnorm(x, t) * scale_e

In [11]:
ddpm

DiffusionModel(
  (net): EBMDiffusionModel(
    (net): Resnet(
      (layer_x): Linear(in_features=23, out_features=128, bias=True)
      (layers): ModuleList(
        (0-3): 4 x Sequential(
          (0): Linear(in_features=128, out_features=256, bias=True)
          (1): Linear(in_features=32, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Linear(in_features=256, out_features=128, bias=True)
        )
      )
      (last_layer): Linear(in_features=128, out_features=23, bias=True)
      (time_emb): Embedding(100, 32)
    )
  )
)

In [12]:
losses = []
test_logpx = []
itr = 0

ddpm.train()
optimizer = torch.optim.Adam(ddpm.parameters(), lr=1e-3)


num_steps = 200000
for itr in range(num_steps):
    ddpm.train()
    
    x = data_sampler.sample(keys=["observations", "actions"], batch_size=batch_size, concat=True)

    start_time = time.time()

    loss = ddpm.loss(x)
    loss = loss.mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    duration_update = time.time() - start_time

    if itr % 100 == 0:
        print(itr, loss, "time:", duration_update)
        losses.append(loss)
        
    if itr % 1000 == 0:
        ddpm.eval()
        x_samp = ddpm.sample(n=1)
        print(x_samp)

0 tensor(30.7890, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.4993598461151123
tensor([[-0.2572, -0.2860,  0.1034,  0.0638,  0.1779,  0.0331,  0.0363,  0.0224,
          0.0909, -0.1637,  0.0379, -0.0761, -0.0349,  0.2749,  0.0080,  0.6047,
         -0.0560,  0.1470,  0.2150, -0.1010,  0.2330,  0.1326,  0.0382]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
100 tensor(0.5459, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006296634674072266
200 tensor(0.4913, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006285190582275391
300 tensor(0.4672, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0062103271484375
400 tensor(0.4534, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006283998489379883
500 tensor(0.4574, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006414175033569336
600 tensor(0.4325, device='cuda:0', dtype=torc

KeyboardInterrupt: 